<a href="https://colab.research.google.com/github/Sarthak9504/SY-Sem1-CourseProjects/blob/EDAI-3/Marathi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [72]:
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report, f1_score
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras.models import Sequential
from keras import layers
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import tensorflow as tf
import re
from google.colab import drive
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional

In [7]:
df=pd.read_excel("/content/tweets-train-cleaned.xlsx", index_col='Unnamed: 0')
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:

df.loc[~df['tweet'].isnull()]

,tweet,label
0,ज्येष्ठ पत्रकार अनंत दीक्षित यां निधन वृत्त दु...,-1
1,सर्वोच् न्ल निर्देश डावलून पुण पोलिस प्र आनंद ...,-1
2,उद्ध ठाकर भाजपासोबत युत शिसैनिक अमान िश्ासघ फस...,-1
3,आपल समाज खूप मोठ त्य घटक अंतर्भूत त या दिव्यां...,1
4,बलात्कार बदल बलात्कार घेतल पाहिज म्हणणार सावरक...,-1
...,...,...
12109,त्याचबरोबर काज व्यावसायिकां मागील कालावध व्हॅट...,1
12110,सरकार कर्जमाफ योज लाग करुन सुमार वर्ष काळ उलटू...,-1
12111,लोकश मतदार खऱ्य अर्थ सत्ताधार असत धमकािण अथ प्...,-1
12112,छत्रपत शिवाज महाराज ंच् नाव शिवसेने स्थाप ंन श...,-1


In [9]:
df.shape

(12114, 2)

In [10]:
df.loc[~df['tweet'].isnull()]

,tweet,label
0,ज्येष्ठ पत्रकार अनंत दीक्षित यां निधन वृत्त दु...,-1
1,सर्वोच् न्ल निर्देश डावलून पुण पोलिस प्र आनंद ...,-1
2,उद्ध ठाकर भाजपासोबत युत शिसैनिक अमान िश्ासघ फस...,-1
3,आपल समाज खूप मोठ त्य घटक अंतर्भूत त या दिव्यां...,1
4,बलात्कार बदल बलात्कार घेतल पाहिज म्हणणार सावरक...,-1
...,...,...
12109,त्याचबरोबर काज व्यावसायिकां मागील कालावध व्हॅट...,1
12110,सरकार कर्जमाफ योज लाग करुन सुमार वर्ष काळ उलटू...,-1
12111,लोकश मतदार खऱ्य अर्थ सत्ताधार असत धमकािण अथ प्...,-1
12112,छत्रपत शिवाज महाराज ंच् नाव शिवसेने स्थाप ंन श...,-1


In [11]:
df_v=pd.read_excel("/content/tweets-valid-cleaned.xlsx", index_col='Unnamed: 0')

In [12]:
df_v.loc[~df_v['tweet'].isnull()]

,tweet,label
0,कोर काळ पंतप्रधान मा नरेंद्र मोदीज य समाज घटक ...,1
1,पुण राष्ट्रसे दल अभ्सिक िभाग ंच् संयुक्त िद्यम...,1
2,पुण क्रांत ज्योत कार्यक्रम जेष्ठ समाजवाद ने मा...,0
3,आद्यक्रांतिकारक उमाजीराज नाईक य जयंतीनिमित्त व...,1
4,महाराष्ट्र मुख्यमंत्र शपथ घेतल्याबद्दल शिवस पक...,1
...,...,...
1495,पोलिसभरत प्रक्रिय जु निकषांनुसार व्व याबाबत मा...,1
1496,निध उद्ध्वस्त कुटुंबांमधील मुल शिक्षण वापरल जा...,1
1497,िकासा फक्त घोषणाबाज जाहिरातबाज राज्य जनते दिशा...,0
1498,विधानसभेत राष्ट्रीय जनगण ओबीसीं इतर मागासवर्गी...,0


In [13]:
df_v.shape

(1500, 2)

In [14]:
df_t = pd.read_excel("/content/tweets-test-cleaned.xlsx", index_col='Unnamed: 0')

In [15]:
df_t.shape

(2250, 2)

In [16]:
max(df_t['tweet'].str.len())

240

In [17]:
max(df_v['tweet'].str.len())

245

In [18]:
num_negative_1 = (df['label'] == -1).sum()
print("Number of data instances under label -1:", num_negative_1)

Number of data instances under label -1: 4038


In [19]:
vocab_df=pd.concat([df_v['tweet'], df.loc[~df['tweet'].isnull(), 'tweet']], axis = 0)

In [20]:
vocab_df.isnull().sum()

0

In [21]:
set2=set()
for row in vocab_df.iteritems():
  for words in row[1].split():
    set2.add(words)
len(set2)

<ipython-input-21-c0937d06b7d9>:2: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for row in vocab_df.iteritems():


38045

In [22]:

vocab_size=40000


In [23]:
from collections import Counter
def counter_word(text_col):
  count=Counter()
  for text in text_col.values:
    for word in text.split():
      count[word]+=1
  return count

In [24]:

counter = counter_word(vocab_df)

len(counter)


38045

In [25]:
df = df[df['label'] != 2]
df_v = df_v[df_v['label'] != 2]
df_t = df_t[df_t['label'] != 2]

In [26]:

from tensorflow.keras.preprocessing.text import Tokenizer
tokinizer=Tokenizer(num_words=40000)
tokinizer.fit_on_texts(df.loc[~df['tweet'].isnull(), 'tweet'])

In [27]:

word_index = tokinizer.word_index

In [28]:
word_index

{'य': 1,
 'सरकार': 2,
 'महाराष्ट्र': 3,
 'यां': 4,
 'आ': 5,
 'राज्य': 6,
 '…': 7,
 'करण्य': 8,
 'भाजप': 9,
 'कर': 10,
 'काँग्रेस': 11,
 'करत': 12,
 'शुभेच्छ': 13,
 'त्य': 14,
 'त': 15,
 'राज': 16,
 'घेतल': 17,
 'उपस्थित': 18,
 'दि': 19,
 'मा': 20,
 'देश': 21,
 'पक्ष': 22,
 'अभिवादन': 23,
 'सुर': 24,
 'मुंबई': 25,
 'विनम्र': 26,
 'श्र': 27,
 'मुख्यमंत्र': 28,
 'भेट': 29,
 'मराठ': 30,
 'करणार': 31,
 'या': 32,
 'शेतकर': 33,
 'शेतकऱ्य': 34,
 'वर्ष': 35,
 'जन': 36,
 'ने': 37,
 'हार्दिक': 38,
 'श्रद्धांजल': 39,
 'माज': 40,
 'निर्णय': 41,
 'सर्व': 42,
 'लोक': 43,
 'मदत': 44,
 'मंत्र': 45,
 'ना': 46,
 'अस': 47,
 'असत': 48,
 'माझ': 49,
 'च्य': 50,
 'मागण': 51,
 'केंद्र': 52,
 'पाहिज': 53,
 'मोद': 54,
 'महाराज': 55,
 'प्रश्न': 56,
 'साहेब': 57,
 'राष्ट्रवाद': 58,
 'अश': 59,
 'चर्': 60,
 'काळ': 61,
 'भावपूर्ण': 62,
 'भाग': 63,
 'करण': 64,
 'समाज': 65,
 'अध्यक्ष': 66,
 'घेत': 67,
 'कोर': 68,
 'मात्र': 69,
 'बैठक': 70,
 'मोठ': 71,
 'जिल्ह्य': 72,
 'पुण': 73,
 'माझ्य': 74,
 'कराव': 75,
 'ंन': 76,
 '

In [29]:
train_sequences = tokinizer.texts_to_sequences(df.loc[~df['tweet'].isnull(), 'tweet'])

In [30]:
test_sequences = tokinizer.texts_to_sequences(df_t['tweet'].values)


In [31]:
val_sequences = tokinizer.texts_to_sequences(df_v['tweet'].values)


In [32]:

print(df['tweet'][0])
print(train_sequences[0])

ज्येष्ठ पत्रकार अनंत दीक्षित यां निधन वृत्त दुखद दशकं परखड लेखण पत्रकारितेत अमूल्य योगदान दि दीक्षित यां मार्गदर्शनाखाल पत्रकारां पिढ घडल अनंत दीक्षित य भावपूर्ण श्रद्धांजल त्य परिवार दुख सहभाग आहोत
[83, 132, 1800, 4171, 4, 77, 795, 1676, 8272, 2692, 2387, 10711, 2693, 268, 19, 4171, 4, 1864, 5150, 994, 1732, 1800, 4171, 1, 62, 39, 14, 1733, 1230, 97, 236]


In [33]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [34]:
train_padded=pad_sequences(train_sequences, maxlen=250, padding='post', truncating='post')
val_padded=pad_sequences(val_sequences, maxlen=250, padding='post', truncating='post')
test_padded=pad_sequences(test_sequences, maxlen=250, padding='post', truncating='post')

In [35]:
print(train_padded.shape, val_padded.shape, test_padded.shape)

(12112, 250) (1500, 250) (2250, 250)


In [36]:
from keras.layers import Embedding, LSTM, Dense, Dropout
from sklearn.metrics import classification_report
from keras.optimizers import Adam
model = Sequential()

In [37]:
model.add(layers.Embedding(40000, 32, input_length=250))
model.add(layers.Dense(256, activation='gelu'))
model.add(layers.Flatten())
model.add(layers.Dense(3, activation='softmax'))

In [38]:
optim = tf.keras.optimizers.Adam(learning_rate=0.0002)
model.compile(loss='categorical_crossentropy', optimizer=optim, metrics=['accuracy'])


In [39]:
train_label_categorical = to_categorical(df.loc[~df['tweet'].isnull(), 'label'], num_classes=3)


val_label_categorical = to_categorical(df_v['label'].values, num_classes=3)


test_label_categorical = to_categorical(df_t['label'].values, num_classes=3)

In [40]:

val_label_categorical

array([[0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.]], dtype=float32)

In [41]:
train_label_categorical

array([[0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       ...,
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.]], dtype=float32)

In [42]:
tf.config.run_functions_eagerly(True)
model.fit(train_padded, to_categorical(df.loc[~df['tweet'].isnull(), 'label'].values, num_classes=3), epochs=5, validation_data=(val_padded, to_categorical(df_v['label'].values, num_classes=3)), verbose=2)

Epoch 1/5


/usr/local/lib/python3.10/dist-packages/tensorflow/python/data/ops/structured_function.py:254: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


379/379 - 21s - loss: 1.0041 - accuracy: 0.4714 - val_loss: 0.8029 - val_accuracy: 0.6313 - 21s/epoch - 55ms/step
Epoch 2/5
379/379 - 18s - loss: 0.6389 - accuracy: 0.7452 - val_loss: 0.5573 - val_accuracy: 0.7700 - 18s/epoch - 47ms/step
Epoch 3/5
379/379 - 14s - loss: 0.3850 - accuracy: 0.8492 - val_loss: 0.5152 - val_accuracy: 0.7867 - 14s/epoch - 36ms/step
Epoch 4/5
379/379 - 14s - loss: 0.2454 - accuracy: 0.9145 - val_loss: 0.5352 - val_accuracy: 0.7787 - 14s/epoch - 36ms/step
Epoch 5/5
379/379 - 14s - loss: 0.1530 - accuracy: 0.9525 - val_loss: 0.5774 - val_accuracy: 0.7880 - 14s/epoch - 36ms/step


In [43]:

model.evaluate(test_padded, to_categorical(df_t['label'].values, num_classes=3))

71/71 [==============================] - 1s 14ms/step - loss: 0.6123 - accuracy: 0.7782


[0.6122838854789734, 0.7782222032546997]

In [44]:
predictions = model.predict(test_padded)
predictions


71/71 [==============================] - 0s 5ms/step


array([[4.1393687e-08, 1.0355506e-07, 9.9999988e-01],
       [8.6170193e-03, 2.9261367e-02, 9.6212155e-01],
       [9.1106771e-04, 5.0790496e-02, 9.4829839e-01],
       ...,
       [9.9272525e-01, 2.0765781e-03, 5.1982128e-03],
       [5.7672334e-01, 3.8663429e-01, 3.6642402e-02],
       [1.5193270e-01, 8.3578342e-01, 1.2283850e-02]], dtype=float32)

In [45]:
import sklearn
matrix = sklearn.metrics.confusion_matrix(test_label_categorical.argmax(axis=1), predictions.argmax(axis=1))
print(matrix)

[[565 102  83]
 [124 569  57]
 [ 95  38 617]]


In [46]:
from sklearn.metrics import accuracy_score, f1_score
print(accuracy_score(test_label_categorical.argmax(axis=1), predictions.argmax(axis=1)))
print(f1_score(test_label_categorical.argmax(axis=1), predictions.argmax(axis=1), average=None))


0.7782222222222223
[0.73663625 0.77998629 0.81884539]


In [47]:
!pip install transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 56.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 31.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 107.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 83.2 MB/s eta 0:00:00


In [48]:
!pip install sentence-transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 10.3 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125924 sha256=6d683028a0a59aa9fa76cfd565847c35bdff8d6af2bd36c42b1250ffa549843c
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [49]:
from transformers import AutoTokenizer, BertTokenizer, TFBertModel, AutoModel, TFAutoModel, TFAutoModelForSequenceClassification

In [50]:
from sentence_transformers import SentenceTransformer

In [51]:
tokenizer = AutoTokenizer.from_pretrained("l3cube-pune/indic-sentence-bert-nli")

In [52]:
df = pd.read_excel("/content/tweets-train-cleaned.xlsx", index_col='Unnamed: 0')
df_v = pd.read_excel("/content/tweets-valid-cleaned.xlsx", index_col='Unnamed: 0')
df_t = pd.read_excel("/content/tweets-test-cleaned.xlsx", index_col='Unnamed: 0')


In [53]:
train_inputs = tokenizer(df.loc[~df['tweet'].isnull(), 'tweet'].tolist(), padding=True, truncation=True, max_length=250, return_tensors="tf")
val_inputs = tokenizer(df_v['tweet'].tolist(), padding=True, truncation=True, max_length=250, return_tensors="tf")
test_inputs = tokenizer(df_t['tweet'].tolist(), padding=True, truncation=True, max_length=250, return_tensors="tf")

In [54]:
train_labels = df.loc[~df['tweet'].isnull(), 'label'].values
val_labels = df_v['label'].values
test_labels = df_t['label'].values

In [55]:
num_classes = 3
train_label_categorical = to_categorical(train_labels, num_classes=num_classes)
val_label_categorical = to_categorical(val_labels, num_classes=num_classes)
test_label_categorical = to_categorical(test_labels, num_classes=num_classes)

In [136]:
train_labels = df.loc[~df['tweet'].isnull(), 'label'].values
train_label_categorical = to_categorical(train_labels, num_classes=3)

In [137]:
val_labels = df.loc[~df['tweet'].isnull(), 'label'].values
val_label_categorical = to_categorical(val_labels, num_classes=3)

In [138]:
aligned_train_label_categorical = train_label_categorical[:1212]

In [56]:
model = SentenceTransformer("l3cube-pune/indic-sentence-bert-nli")

In [57]:
train_embeddings = model.encode(df.loc[~df['tweet'].isnull(), 'tweet'].tolist(), show_progress_bar=True)
val_embeddings = model.encode(df_v['tweet'].tolist(), show_progress_bar=True)
test_embeddings = model.encode(df_t['tweet'].tolist(), show_progress_bar=True)

Batches:   0%|          | 0/379 [00:00<?, ?it/s]

Batches:   0%|          | 0/47 [00:00<?, ?it/s]

Batches:   0%|          | 0/71 [00:00<?, ?it/s]

In [58]:
classification_model = tf.keras.Sequential([
    Dense(256, activation='relu', input_shape=(train_embeddings.shape[1],)),
    Dropout(0.2),
    Dense(num_classes, activation='softmax')
])

In [59]:
optim = tf.keras.optimizers.Adam(learning_rate=0.0002)
classification_model.compile(loss='categorical_crossentropy', optimizer=optim, metrics=['accuracy'])


In [60]:
classification_model.fit(
    train_embeddings,
    train_label_categorical,
    epochs=9,
    batch_size=32,
    validation_data=(val_embeddings, val_label_categorical),
    verbose=2
)

Epoch 1/9


/usr/local/lib/python3.10/dist-packages/tensorflow/python/data/ops/structured_function.py:254: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


379/379 - 16s - loss: 0.8741 - accuracy: 0.6911 - val_loss: 0.6791 - val_accuracy: 0.7433 - 16s/epoch - 41ms/step
Epoch 2/9
379/379 - 14s - loss: 0.6548 - accuracy: 0.7340 - val_loss: 0.5986 - val_accuracy: 0.7593 - 14s/epoch - 36ms/step
Epoch 3/9
379/379 - 11s - loss: 0.6165 - accuracy: 0.7439 - val_loss: 0.5788 - val_accuracy: 0.7640 - 11s/epoch - 30ms/step
Epoch 4/9
379/379 - 11s - loss: 0.6030 - accuracy: 0.7479 - val_loss: 0.5708 - val_accuracy: 0.7633 - 11s/epoch - 30ms/step
Epoch 5/9
379/379 - 10s - loss: 0.5925 - accuracy: 0.7523 - val_loss: 0.5626 - val_accuracy: 0.7747 - 10s/epoch - 26ms/step
Epoch 6/9
379/379 - 11s - loss: 0.5841 - accuracy: 0.7558 - val_loss: 0.5563 - val_accuracy: 0.7767 - 11s/epoch - 29ms/step
Epoch 7/9
379/379 - 14s - loss: 0.5783 - accuracy: 0.7608 - val_loss: 0.5510 - val_accuracy: 0.7713 - 14s/epoch - 36ms/step
Epoch 8/9
379/379 - 12s - loss: 0.5723 - accuracy: 0.7633 - val_loss: 0.5486 - val_accuracy: 0.7753 - 12s/epoch - 32ms/step
Epoch 9/9
379/379 

In [61]:
test_loss, test_acc = classification_model.evaluate(test_embeddings, test_label_categorical)
print("Test accuracy:", test_acc)

71/71 [==============================] - 1s 12ms/step - loss: 0.5634 - accuracy: 0.7716
Test accuracy: 0.7715555429458618


In [62]:
predictions = classification_model.predict(test_embeddings)
predicted_labels = np.argmax(predictions, axis=1)
predictions

71/71 [==============================] - 0s 3ms/step


array([[0.03121307, 0.04034865, 0.9284383 ],
       [0.15382148, 0.05401564, 0.7921629 ],
       [0.20654057, 0.54907703, 0.24438243],
       ...,
       [0.5224558 , 0.08122341, 0.39632076],
       [0.774198  , 0.11040504, 0.11539695],
       [0.12229969, 0.84144443, 0.03625585]], dtype=float32)

In [63]:
print(classification_report(test_labels, predicted_labels))

              precision    recall  f1-score   support

          -1       0.00      0.00      0.00       750
           0       0.73      0.74      0.73       750
           1       0.79      0.75      0.77       750
           2       0.00      0.00      0.00         0

    accuracy                           0.50      2250
   macro avg       0.38      0.37      0.38      2250
weighted avg       0.51      0.50      0.50      2250



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.

In [161]:
matrix = confusion_matrix(test_label_categorical.argmax(axis=1), predictions.argmax(axis=1))
print(matrix)

[[575  80  95]
 [140 547  63]
 [ 95  34 621]]


In [71]:
from sklearn.metrics import accuracy_score, f1_score
print(accuracy_score(test_label_categorical.argmax(axis=1), predictions.argmax(axis=1)))
print(f1_score(test_label_categorical.argmax(axis=1), predictions.argmax(axis=1), average=None))

0.7715555555555556
[0.73293572 0.77039068 0.81070496]


Bi-LSTM

In [82]:
train_embeddings = model.encode(df.loc[~df['tweet'].isnull(), 'tweet'].tolist(), show_progress_bar=True)
val_embeddings = model.encode(df_v['tweet'].tolist(), show_progress_bar=True)
test_embeddings = model.encode(df_t['tweet'].tolist(), show_progress_bar=True)

Batches:   0%|          | 0/379 [00:00<?, ?it/s]

Batches:   0%|          | 0/47 [00:00<?, ?it/s]

Batches:   0%|          | 0/71 [00:00<?, ?it/s]

In [83]:
embedding_dim = 768

In [96]:
def convert_to_sequences(embeddings, sequence_length):
    num_embeddings, embedding_dim = embeddings.shape
    sequences = []

    for i in range(0, num_embeddings, sequence_length):
        sequence = embeddings[i:i+sequence_length]
        sequences.append(sequence)

    return np.array(sequences)

In [160]:
sequence_length = 10
train_sequences = convert_to_sequences(train_embeddings, sequence_length)
val_sequences = convert_to_sequences(val_embeddings, sequence_length)
test_sequences = convert_to_sequences(test_embeddings, sequence_length)

<ipython-input-96-964fb3420b0c>:9: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(sequences)


In [126]:
classification_model = tf.keras.Sequential([
    Bidirectional(LSTM(128, return_sequences=True), input_shape=(sequence_length, embedding_dim)),
    Dense(256, activation='relu'),
    Dropout(0.2),
    Dense(num_classes, activation='softmax')
])

In [141]:
max_sequence_length = 10
train_padded_sequences = pad_sequences(train_sequences, maxlen=max_sequence_length, padding='post', truncating='post')
val_padded_sequences = pad_sequences(val_sequences, maxlen=max_sequence_length, padding='post', truncating='post')
test_padded_sequences = pad_sequences(test_sequences, maxlen=max_sequence_length, padding='post', truncating='post')

In [162]:
print("Train data shapes:")
print(train_padded_sequences.shape)
print(train_label_categorical.shape)

Train data shapes:
(49, 250, 768)
(12112, 3)


In [157]:
print("Validation data shapes:")
print(val_padded_sequences.shape)
print(val_label_categorical.shape)

Validation data shapes:
(150, 10, 768)
(12112, 3)


In [158]:

print("Test data shapes:")
print(test_padded_sequences.shape)
print(test_label_categorical.shape)

Test data shapes:
(225, 10, 768)
(2250, 3)


In [150]:
classification_model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_5 (Bidirectio  (None, 10, 256)          918528    
 nal)                                                            
                                                                 
 dense_16 (Dense)            (None, 10, 256)           65792     
                                                                 
 dropout_7 (Dropout)         (None, 10, 256)           0         
                                                                 
 dense_17 (Dense)            (None, 10, 3)             771       
                                                                 
Total params: 985,091
Trainable params: 985,091
Non-trainable params: 0
_________________________________________________________________


In [129]:
optim = tf.keras.optimizers.Adam(learning_rate=0.0002)
classification_model.compile(loss='categorical_crossentropy', optimizer=optim, metrics=['accuracy'])

In [155]:
classification_model.fit(
    aligned_train_sequences,
    aligned_train_label_categorical,
    epochs=9,
    batch_size=32,
    validation_data=(val_padded_sequences, val_label_categorical),
    verbose=1
)

ValueError: ignored

In [77]:
test_loss, test_acc = classification_model.evaluate(test_embeddings, test_label_categorical)
print("Test accuracy:", test_acc)

71/71 [==============================] - 3s 42ms/step - loss: 0.5574 - accuracy: 0.7747
Test accuracy: 0.7746666669845581


In [78]:
predictions = classification_model.predict(test_embeddings)
predicted_labels = np.argmax(predictions, axis=1)
predictions

71/71 [==============================] - 0s 6ms/step


array([[0.03194045, 0.02768869, 0.9403708 ],
       [0.16699283, 0.07405228, 0.7589548 ],
       [0.20820762, 0.56400704, 0.22778538],
       ...,
       [0.57902706, 0.11000835, 0.31096455],
       [0.9065635 , 0.0523167 , 0.04111969],
       [0.08805405, 0.8879048 , 0.02404109]], dtype=float32)

In [79]:
print(classification_report(test_labels, predicted_labels))

              precision    recall  f1-score   support

          -1       0.00      0.00      0.00       750
           0       0.71      0.77      0.74       750
           1       0.83      0.73      0.78       750
           2       0.00      0.00      0.00         0

    accuracy                           0.50      2250
   macro avg       0.38      0.37      0.38      2250
weighted avg       0.51      0.50      0.50      2250



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.

In [80]:
matrix = confusion_matrix(test_label_categorical.argmax(axis=1), predictions.argmax(axis=1))
print(matrix)

[[575  80  95]
 [140 547  63]
 [ 95  34 621]]


In [81]:
print(accuracy_score(test_label_categorical.argmax(axis=1), predictions.argmax(axis=1)))
print(f1_score(test_label_categorical.argmax(axis=1), predictions.argmax(axis=1), average=None))

0.7746666666666666
[0.73717949 0.77533664 0.81229562]
